In [ ]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy import misc
from config import settings, settings2, settings3
from image_search import *
import os, urllib2

In [ ]:
image_searcher = ImageSearcher('../models/my-model/original_deploy.prototxt', '../models/caffenet-yos/caffenet-yos-weights', settings)
image_searcher_sparse = ImageSearcher('../models/my-model/original_deploy.prototxt', '/home/ubuntu/sdg/sparse_net/sparse_solver.caffemodel', settings2)

In [ ]:
# Random Image list
example_list = ['http://pic29.nipic.com/20130506/3822951_102752055000_2.jpg',
                'http://www.yangfenzi.com/wp-content/uploads/2016/03/kf-320x320.jpg',
                'http://image97.360doc.com/DownloadImg/2016/05/1921/72180400_1.jpg',
                'http://pic23.nipic.com/20120917/3646857_190502244141_2.jpg',
                'http://pic27.nipic.com/20130331/12213721_235901077001_2.jpg',
                'http://pic25.nipic.com/20121120/11287584_092640701115_2.jpg',
                'http://pic3.zhongsou.com/image/380af8d631efe8e9768.jpg', 
                'http://ww2.sinaimg.cn/bmiddle/9da3c13ejw1edywcwmcuhj20fm09yweu.jpg']

if not os.path.isdir('temp_image'):
    os.mkdir('temp_image')
for index in range(len(example_list)):
    temp_path = 'temp_image/temp' + str(index) + '.jpg'
    img = urllib2.urlopen(example_list[index])
    with open(temp_path, 'w') as f:
        f.write(img.read())
    print("Your query Image")
    plt.imshow(misc.imread(temp_path))
    plt.show()

In [ ]:
print("Query results:")
temp_path = 'temp_image/temp' + str(7) + '.jpg'
results = image_searcher_sparse.query_image(temp_path)
for i in range(0, 10):
    plt.imshow(misc.imread(results[i]))
    plt.show()

In [ ]:
sparse_result = ['1111011100',
'1000100111',
'1000000000',
'0000110001',
'1111111001',
'0000010000',
'0001000000',
'1100000000']

regular_result = ['1100010111',
'1000000001',
'1000000000',
'0101000100',
'0110000001',
'1000000001',
'0010110000',
'0000000000']

In [ ]:
# Compute precision-recall curve
def get_precision(result_list):
    correct_list = [0] * 10
    for res in result_list:
        for i in range(0, 10):
            if res[i] == '1':
                correct_list[i] += 1
    precision = []
    for i in range(10):
        if i == 0:
            precision.append(correct_list[i])
        else:
            precision.append(precision[-1] + correct_list[i])
    for i in range(10):
        precision[i] /= 10.0 * (i+1)
    return precision
plt.plot(get_precision(sparse_result), label='Sparse CaffeNet')
plt.plot(get_precision(regular_result), hold=True, label='CaffeNet')
plt.xlabel('number of results')
plt.ylabel('precision')
plt.legend()
plt.show()